# Analyse des temps de paroles des élections europeenes

Preparations des fichiers pour affichage.<br>
<br>
Le site de l'arcom nous fournie plusieurs fichiers à l'adresse suivante : <br>
<a href = "https://www.arcom.fr/actualites/elections-europeennes-2024-publication-des-releves-des-temps-de-parole"> page arcom </a><br>


In [1]:
# import des bibliotheques
import pandas as pd
import os


In [2]:
#lister les nom des fichiers dans les données brutes
liste_fichier =os.listdir('donnee_arcom_EU')


In [3]:
# faire les trucs pour tout les fichiers :
temps_parole_EU2024 = pd.DataFrame()
for element in liste_fichier:
    # lire le fichier 
    tempo_df = pd.read_csv('donnee_arcom_EU/'+element,encoding='ISO-8859-3',sep=';',header=3)
    # garder les 2 derniere ligne du temps presidentiel
    df_president = tempo_df.tail(2)
    # supprimer les ligne du temps presidentiel
    tempo_df = tempo_df[:-2]
    # rajouter une ligne pour le temps presidentiel 
    tempo_df.loc[len(tempo_df)]=['temps_president',df_president.iat[1,1]]
    # rajouter le nom de la chaine
    tempo_df['Nom du media'] = (element[30:-4])
    tempo_df = tempo_df[tempo_df['Listes candidates']!='Total des temps de parole']
    # concat les dataframe entre eux
    temps_parole_EU2024 = pd.concat([temps_parole_EU2024,tempo_df])
    

In [4]:
couleur_pol = pd.read_csv('donnee_contruite/couleurs.csv',encoding='ISO-8859-3')

Liste des medias contenante les fichiers :<br>
Canal plus n'a pas fourni de temps de paroles<br>

In [5]:
print(temps_parole_EU2024['Nom du media'].unique())

['BFMBusiness' 'BFMTV' 'C8' 'Cnews' 'Euronews' 'Europe1' 'France2'
 'France24' 'France3' 'France5' 'FranceCulture' 'FranceInfo' 'FranceInter'
 'LCI' 'M6' 'RFI' 'RMC-Story' 'RMC' 'RMCDecouverte' 'RTL' 'RadioClassique'
 'SudRadio' 'TF1' 'TMC' 'TV5Monde' 'franceinfo_']


In [6]:
temps_parole_EU2024['Listes candidates'].unique()

array(['temps_president', 'ALLIANCE RURALE', 'BESOIN D?EUROPE',
       'CHANGER L?EUROPE', 'DEFENDRE LES ENFANTS',
       'DEMOCRATIE REPRESENTATIVE', 'ÉCOLOGIE AU CENTRE',
       'ECOLOGIE POSITIVE ET TERRITOIRES',
       'ÉQUINOXE: ÉCOLOGIE PRATIQUE ET RENOUVEAU DÉMOCRATIQUE',
       'ESPERANTO LANGUE COMMUNE', 'EUROPE ECOLOGIE',
       'EUROPE TERRITOIRES ÉCOLOGIE',
       'FORTERESSE EUROPE - LISTE D?UNITE NATIONALISTE', 'FRANCE LIBRE',
       'FREE PALESTINE',
       'GAUCHE UNIE POUR LE MONDE DU TRAVAIL SOUTENUE PAR FABIEN ROUSSEL',
       'LA DROITE POUR FAIRE ENTENDRE LA VOIX DE LA FRANCE EN EUROPE',
       'LA FRANCE FIERE, MENEE PAR MARION MARECHAL ET SOUTENUE PAR ÉRIC ZEMMOUR',
       'LA FRANCE INSOUMISE - UNION POPULAIRE',
       'LA FRANCE REVIENT! AVEC JORDAN BARDELLA ET MARINE LE PEN',
       'LA RUCHE CITOYENNE', 'LIBERTE DEMOCRATIQUE FRANCAISE',
       'LISTE ASSELINEAU-FREXIT, POUR LE POUVOIR D?ACHAT ET POUR LA PAIX',
       'LUTTE OUVRIERE LE CAMP DES TRAVAILLEURS',

In [7]:
# listez les media en fonction de leur categories
liste_tv_gen = ['TMC', 'France2', 'France3','M6', 'RMC-Story',
       'TF1', 'France5', 'C8', 'RMCDecouverte', 'TV5Monde',
       ]

liste_tv_info = ['LCI', 'franceinfo_',
       'BFMTV', 'France24', 'Euronews', 'Cnews']

liste_radio = ['RTL', 'RFI', 'FranceInfo', 'FranceCulture',
       'SudRadio', 'RMC', 'Europe1', 'BFMBusiness',
       'RadioClassique', 'FranceInter']

# ajouter les categories dans le dataframe :
temps_parole_EU2024['Type de media'] = 'inconnu'

for media in liste_tv_gen: 
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'TV généraliste'
for media in liste_tv_info:
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'TV info continue' 
for media in liste_radio:
       temps_parole_EU2024.loc[temps_parole_EU2024['Nom du media']==media,'Type de media'] = 'radio' 

Catégorisation des médias : <br>

In [8]:
temps_parole_EU2024['Type de media'].unique()

array(['radio', 'TV info continue', 'TV généraliste'], dtype=object)

In [9]:
temps_parole_EU2024.info()

<class 'pandas.core.frame.DataFrame'>
Index: 711 entries, 0 to 40
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Listes candidates      711 non-null    object
 1   Total période (durée)  711 non-null    object
 2   Nom du media           711 non-null    object
 3   Type de media          711 non-null    object
dtypes: object(4)
memory usage: 27.8+ KB


In [10]:
# voir pour changer le type de données de la periode tout convertir en seconde puis en heure
temps_parole_EU2024['heure'] = temps_parole_EU2024['Total période (durée)'].str[:2]
temps_parole_EU2024['minute'] = temps_parole_EU2024['Total période (durée)'].str[3:5]
temps_parole_EU2024['seconde'] = temps_parole_EU2024['Total période (durée)'].str[-2:]

temps_parole_EU2024['heure'] = temps_parole_EU2024['heure'].astype(int)
temps_parole_EU2024['minute'] = temps_parole_EU2024['minute'].astype(int)
temps_parole_EU2024['seconde'] = temps_parole_EU2024['seconde'].astype(int)

temps_parole_EU2024['Durée en seconde'] = temps_parole_EU2024['heure']*3600 + temps_parole_EU2024['minute']*60 +temps_parole_EU2024['seconde']
temps_parole_EU2024['Durée en minute'] = temps_parole_EU2024['Durée en seconde']/60 

In [11]:
temps_parole_EU2024.drop(columns=['heure','minute','seconde'],inplace=True)

# Temps de parole en secondes des listes :<br>

In [12]:
temps_parole_EU2024.groupby(by='Listes candidates').agg({'Durée en seconde':'sum'}).reset_index()

,Listes candidates,Durée en seconde
0,ALLIANCE CENTRISTE ET TERRITOIRES EN MOUVEMENT,194
1,ALLIANCE RURALE,9592
2,BESOIN D?EUROPE,184834
3,CHANGER L?EUROPE,7712
4,DEFENDRE LES ENFANTS,775
5,DEMOCRATIE REPRESENTATIVE,461
6,ECOLOGIE POSITIVE ET TERRITOIRES,1369
7,ESPERANTO LANGUE COMMUNE,949
8,EUROPE ECOLOGIE,79731
9,EUROPE EQUITABLE,489


In [13]:
temps_parole_EU2024.groupby(by='Nom du media').agg({'Durée en seconde':'sum'}).reset_index()

,Nom du media,Durée en seconde
0,BFMBusiness,270
1,BFMTV,159919
2,C8,45402
3,Cnews,94860
4,Euronews,6254
5,Europe1,74632
6,France2,46900
7,France24,28061
8,France3,14747
9,France5,30607


In [14]:
# calcul du temps total :

df_temps_total = temps_parole_EU2024.groupby(by='Nom du media').agg({'Durée en seconde':'sum'}).reset_index().rename(columns={'Durée en seconde':'Temps total'})

In [15]:
temps_parole_EU2024 = pd.merge(temps_parole_EU2024,df_temps_total,on='Nom du media',how='inner')
temps_parole_EU2024 = pd.merge(temps_parole_EU2024,couleur_pol,on='Listes candidates',how='inner')

In [16]:
# calcul des pourcentages des interventions par chaines

temps_parole_EU2024['Taux temps de parole'] = temps_parole_EU2024['Durée en seconde'] / temps_parole_EU2024['Temps total'] *100

# verification opération

temps_parole_EU2024.groupby(by='Nom du media').agg({'Taux temps de parole':'sum'}).reset_index()

,Nom du media,Taux temps de parole
0,BFMBusiness,100.0
1,BFMTV,100.0
2,C8,100.0
3,Cnews,100.0
4,Euronews,100.0
5,Europe1,100.0
6,France2,100.0
7,France24,100.0
8,France3,100.0
9,France5,100.0


In [17]:
temps_parole_EU2024['Durée en heure'] = temps_parole_EU2024['Durée en seconde']/3600

In [18]:
#bon je change le franceinfo par france info TV pour eviter les confusion entre la TV et la radio
temps_parole_EU2024['Nom du media'] = temps_parole_EU2024['Nom du media'].replace({'franceinfo_':'France info TV'})

In [19]:
# ajout proprio au dataset
media_proprio = pd.read_csv('donnee_contruite/media.csv',encoding='ISO-8859-3')

In [20]:
temps_parole_EU2024 = pd.merge(temps_parole_EU2024,media_proprio,on='Nom du media',how='inner')

In [21]:
temps_parole_EU2024.to_csv('donnee_contruite/temps_paroles_EU2024.csv',index=False)

In [22]:
fichier =pd.read_csv('donnee_contruite/temps_paroles_EU2024.csv')

In [23]:
fichier.head()

,Listes candidates,Total période (durée),Nom du media,Type de media,Durée en seconde,Durée en minute,Temps total,Couleur,Taux temps de parole,Durée en heure,proprio du media
0,temps_president,00:04:30,BFMBusiness,radio,270,4.500000,270,Macron parle,100.000000,0.075000,Altice
1,ALLIANCE RURALE,00:20:23,BFMTV,TV info continue,1223,20.383333,159919,Autres,0.764762,0.339722,Altice
2,BESOIN D?EUROPE,07:55:17,BFMTV,TV info continue,28517,475.283333,159919,Centriste et Liberaux,17.832153,7.921389,Altice
3,CHANGER L?EUROPE,00:19:42,BFMTV,TV info continue,1182,19.700000,159919,Sociaux Democrate,0.739124,0.328333,Altice
4,DEFENDRE LES ENFANTS,00:02:05,BFMTV,TV info continue,125,2.083333,159919,Autres,0.078165,0.034722,Altice


In [24]:
fichier.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 658 entries, 0 to 657
Data columns (total 11 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Listes candidates      658 non-null    object 
 1   Total période (durée)  658 non-null    object 
 2   Nom du media           658 non-null    object 
 3   Type de media          658 non-null    object 
 4   Durée en seconde       658 non-null    int64  
 5   Durée en minute        658 non-null    float64
 6   Temps total            658 non-null    int64  
 7   Couleur                658 non-null    object 
 8   Taux temps de parole   658 non-null    float64
 9   Durée en heure         658 non-null    float64
 10  proprio du media       658 non-null    object 
dtypes: float64(3), int64(2), object(6)
memory usage: 56.7+ KB


In [25]:
fichier.describe()

,Durée en seconde,Durée en minute,Temps total,Taux temps de parole,Durée en heure
count,658.000000,658.000000,658.000000,658.000000,658.000000
mean,1490.425532,24.840426,51798.487842,3.495441,0.414007
std,3800.189248,63.336487,46354.667639,6.989492,1.055608
min,4.000000,0.066667,270.000000,0.008529,0.001111
25%,30.000000,0.500000,14543.000000,0.091446,0.008333
50%,118.500000,1.975000,34440.000000,0.531781,0.032917
75%,873.500000,14.558333,74632.000000,3.771270,0.242639
max,39554.000000,659.233333,159919.000000,100.000000,10.987222
